In [ ]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute

cluster_name = "Wilkens-Cluster"

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)

except:
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_D2_V2', max_nodes=4)
    
    compute_target= ComputeTarget.create(ws,cluster_name,compute_config)
    
compute_target.wait_for_completion(show_output=True)



In [ ]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
### YOUR CODE HERE ###
ps = RandomParameterSampling(
    {
"--C" : choice(0.5,1.0,1.5) ,     
"--max_iter" : choice(50,100,150)
}
)  

# Specify a Policy
### YOUR CODE HERE ###
policy = BanditPolicy(evaluation_interval=1, slack_factor=0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
#https://docs.microsoft.com/en-us/azure/machine-learning/how-to-migrate-from-estimators-to-scriptrunconfig

curated_env_name = 'AzureML-sklearn-0.24-ubuntu18.04-py37-cpu'
train_env = Environment.get(workspace=ws, name=curated_env_name)

#compute_target = ws.compute_targets['my-cluster']
src = ScriptRunConfig(source_directory='.',
                      script='train.py',
                      compute_target=compute_target,
                      environment=train_env)

#sklearn_est = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')
#sklearn_est = 
# Create a ScriptRunConfig Object to specify the configuration details of your training job
### YOUR CODE HERE ###
#src = ScriptRunConfig(source_directory=".",script='train.py',compute_target=cluster_name)

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
### YOUR CODE HERE ###

hyperdrive_config = HyperDriveConfig(
        hyperparameter_sampling=ps,
        primary_metric_name='accuracy',
        run_config=src,
        primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
        max_total_runs = 2, 
        max_concurrent_runs = 4,
        policy=policy)

In [ ]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###

hyperdrive_run=exp.submit(hyperdrive_config)

RunDetails(hyperdrive_run).show()

#hyperdrive_run.get_status()
hyperdrive_run.wait_for_completion(show_output=True)

In [ ]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics() 
##CW
print("Best Run:", best_run.id)
print("\n Accuracy:", best_run_metrics['accuracy'])
print("\n File Name :",best_run.get_file_names())
#print("\n best run details ", best_run.get_details())
#print("\n best run metrics :",best_run_metrics)

# Save Model

model = best_run.register_model(model_name='hyperdrive_best', tags=best_run.best_run_metrics,model_path='outputs/model.joblib')

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
data_url  = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds = TabularDatasetFactory.from_delimited_files(data_url)

In [ ]:
from train import clean_data
from sklearn.model_selection import train_test_split
import pandas as pd


# Use the clean_data function to clean your data.
x, y = clean_data(ds)

x_train, x_test, y_train, y_test = train_test_split(x,y,test_size = .20, random_state = 10)
train_x_y = pd.concat([x_train,y_train], axis=1)

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=train_x_y,
    label_column_name='y',
    enable_onnx_compatible_models=True,
    n_cross_validations=2)



In [2]:
# Submit your automl run

### YOUR CODE HERE ###
automl_run = exp.submit(config= automl_config, show_output= True)

In [ ]:
# Retrieve and save your best automl model.
from azureml.automl.runtime.onnx_convert import OnnxConverter
### YOUR CODE HERE ###
best_run, best_model = automl_run.get_output(return_onnx_model=True)

#Save using Onnx converter
OnnxConverter.save_onnx_model(best_model, './outputs/best_model.onnx')

print(best_model, 'steps')
print("\n")
print(best_run.get_metrics())



In [ ]:
#Delete Model
compute_target.delete()
